In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import time
import tqdm
import glob
import torch
import librosa
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.signal

import dcase_dataset
import sed_utils
import models
import explore
import post_processing as pp
import evaluate_model
import dcase_evaluation
import stats_utils
import copy

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

model = models.get_model('resnet', 48, 16, 1024, 64, 1)
print("Number of parameters: ", count_parameters(model))


# Ensamble + Adaptive network predictions

In [ ]:
def ensemble_prediction(csv_path, experiment_dir, tfs=[], runs=[], thrs=[], adaptive_embedding=False, window_size=None, hop_size_fraction=2):
    n_shot = 5
    
    window_sizes = np.array([2048, 4096, 8192, 16384])
    #window_sizes = np.array([4096, 8192, 16384])

    
    # use correct model
    if adaptive_embedding:
        n_shot_event_lengths = stats_utils.get_nshot_event_lengths(n_shot, csv_path)
        average_event_length = np.min(n_shot_event_lengths) / 2
        average_event_size = int(sample_rate * average_event_length)
        window_size = window_sizes[np.argmin(np.sqrt(np.power(window_sizes-average_event_size, 2)))]
        hop_size = window_size // hop_size_fraction
    else:
        window_size = window_size #window_sizes[2]
        hop_size    = window_size // hop_size_fraction
        #window_size = valid_conf['window_size']
        
    #print("csv_path = {}, window_size = {}, hop_size = {}".format(csv_path, window_size, hop_size))
    
    # TODO: rewrite this a bit to loop over tfs and runs and create paths
    experiment_paths = []
    for tf in tfs:
        for run in runs:
            experiment_path = os.path.join(experiment_dir, tf, 'window_size_{}'.format(window_size), 'run_{}'.format(run))
            experiment_paths.append(experiment_path)
    #print(experiment_paths)
    
    y_predss = []
    for experiment_path in experiment_paths: # ensamble predictions
        # load the predictions
        basename        = os.path.basename(csv_path).split('.')[0]
        prediction_path = os.path.join(experiment_path, 'predictions', '{}_predictions_hop_size_{}.npy'.format(basename, hop_size))
        times_path      = os.path.join(experiment_path, 'predictions', '{}_times_hop_size_{}.npy'.format(basename, hop_size))

        if os.path.exists(prediction_path):
            y_preds = np.load(prediction_path)
            q_times = np.load(times_path)
        else:
            raise ValueError("predictions are missing: ", prediction_path)
        y_predss.append(np.expand_dims(np.array(y_preds), axis=0))
    y_predss = np.concatenate(y_predss, axis=0)
    y_preds  = np.mean(y_predss, axis=0)
    
    sorted_predicitions, sorted_intervals = zip(*sorted(list(zip(y_preds, q_times)), key=lambda x: x[1][0]))
    
    ann_df = pd.read_csv(csv_path)
    ann_df = ann_df.sort_values(by='Starttime', axis=0, ascending=True)
    nth_event = evaluate_model.select_nth_event_with_value(ann_df, 5, value='POS')
    skiptime = nth_event['Endtime']
    
    # evaluate for different thresholds
    set_pos_events = {}
    for thr in thrs:
        set_pos_events[thr] = []

    for thr in thrs:
        binary_predictions = np.zeros(len(sorted_predicitions))

        for idx in range(len(sorted_intervals)):
            y_proba = sorted_predicitions[idx]
            if y_proba > thr:
                binary_predictions[idx] = 1

        # apply a median filter
        binary_predictions = scipy.signal.medfilt(binary_predictions, 1)

        for pred, interval in zip(binary_predictions, sorted_intervals):
            if pred == 1 and not interval[0] < skiptime:
                    filename = os.path.basename(csv_path).replace('.csv', '.wav')
                    set_pos_events[thr].append({
                        'Audiofilename' : filename,
                        'Starttime'     : interval[0],
                        'Endtime'       : interval[1],
                    })
    
    return set_pos_events

def ensemble_set_prediction(csv_paths, experiment_dir, tfs, runs, thrs, adaptive_embedding=False, window_size=None, hop_size_fraction=2):
    set_pos_events = {}
    for thr in thrs:
        set_pos_events[thr] = []

    for csv_path in csv_paths:
        #print(csv_path)
        csv_pos_events = ensemble_prediction(
            csv_path       = csv_path, 
            experiment_dir = experiment_dir, 
            tfs            = tfs,
            runs           = runs,
            thrs           = thrs, 
            adaptive_embedding=adaptive_embedding,
            window_size    = window_size,
            hop_size_fraction=hop_size_fraction,
        )
        #print(csv_pos_events)

        for thr in thrs:
            for pos_event in csv_pos_events[thr]:
                set_pos_events[thr].append(pos_event)
    return set_pos_events

In [ ]:
def compute_scores(set_pos_events, thrs, sets, measures, prediction_path='test_pred_post_csv', run_evaluation=True):
    set_scores = {}
    for s in sets:
        set_scores[s] = {}
        for measure in measures:
            set_scores[s][measure] = []
            
    for thr in thrs:
        pos_events = set_pos_events[thr]

        y_pred_df = pd.DataFrame(pos_events)
        y_pred_df = pp.merge_predictions(y_pred_df)
        y_pred_df = pp.adaptive_remove_unmatchable_predictions(y_pred_df, csv_paths, 5)

        #pred_file_path = 'test_pred_post.csv'
        pred_file_path = prediction_path
        y_pred_df.to_csv(pred_file_path, index=False)
        
        if not os.path.exists('./evals'):
            os.makedirs('./evals')

        if run_evaluation:
            overall_scores, scores_per_subset = dcase_evaluation.evaluate(
                    pred_file_path = pred_file_path,
                    ref_file_path  = root_dir,
                    team_name      = "TeamGBG",
                    dataset        = 'VAL',
                    savepath       = './evals',
                    metadata       = [],
                    verbose        = False #True
            )

            for k in sets:
                for measure in measures:
                    if k == 'all':
                        score = overall_scores[measure]
                    else:
                        score = scores_per_subset[k][measure]
                    set_scores[k][measure].append(score)
    return set_scores

def evaluate_and_plot(configs, csv_paths, fig_path, prediction_path='test_pred_post_csv', run_evaluation=True):
    fig = plt.figure(figsize=(4, 3))
    for config in configs:
        thrs = config['thrs']
        runs = config['runs']
        tfs  = config['tfs']
        adaptive_embedding = config['adaptive_embedding']
        experiment_dir = config['experiment_dir']
        hop_size_fraction=config['hop_size_fraction']
        
        if not adaptive_embedding:
            window_size = config['window_size']
        else:
            window_size = None

        set_pos_events = ensemble_set_prediction(
            csv_paths          = csv_paths,
            experiment_dir     = experiment_dir,
            tfs                = tfs,
            runs               = runs,
            thrs               = thrs,
            adaptive_embedding = adaptive_embedding,
            window_size        = window_size,
            hop_size_fraction  = hop_size_fraction,
        )

        #print(set_pos_events)

        sets = ['HB', 'PB', 'ME', 'all']
        measures = ['f-measure', 'precision', 'recall']
        set_scores = compute_scores(set_pos_events, thrs, sets, measures, prediction_path=prediction_path, run_evaluation=run_evaluation)

        #plt.rcParams['text.usetex'] = True
        
        plt.rc('font', family='serif')
        plt.rc('xtick', labelsize='x-small')
        plt.rc('ytick', labelsize='x-small')

        if run_evaluation:
            plt.plot(thrs, set_scores['all']['f-measure'], label=config['label'])
            plt.xlabel('Threshold')
            plt.ylabel('F-score')
            print("{} max f-measure = {}".format(config['label'], np.max(set_scores['all']['f-measure'])))
    plt.legend()
    plt.savefig(fig_path)
    return fig

# Figure 2

In [ ]:
sample_rate = 22050
root_dir = '/mnt/storage_1/datasets/bioacoustics_dcase2022/Development_Set_{}Hz/Validation_Set/'.format(sample_rate)
csv_paths_me = glob.glob(os.path.join(root_dir, 'ME/*.csv'))
csv_paths_pb = glob.glob(os.path.join(root_dir, 'PB/*.csv'))
csv_paths_hb = glob.glob(os.path.join(root_dir, 'HB/*.csv'))
csv_paths = csv_paths_me + csv_paths_hb + csv_paths_pb

configs = [
    {
        'thrs'               : [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
        'runs'               : [0,1,2,3,4],
        'tfs'                : ['tf_pcen_biodiversity'],
        'adaptive_embedding' : True,
        'stats_group'        : 0,
        'label'              : 'Ensemble',
        'experiment_dir'     : '../experiments/final_ensemble/',
        'hop_size_fraction'  : 2,
    },
    {
        'thrs'               : [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
        'runs'               : [0],
        'tfs'                : ['tf_pcen_biodiversity'],
        'adaptive_embedding' : True,
        'stats_group'        : 0,
        'label'              : 'Prediction 0',
        'experiment_dir'     : '../experiments/final_ensemble/',
        'hop_size_fraction'  : 2,
    },
    {
        'thrs'               : [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
        'runs'               : [1],
        'tfs'                : ['tf_pcen_biodiversity'],
        'adaptive_embedding' : True,
        'stats_group'        : 0,
        'label'              : 'Prediction 1',
        'experiment_dir'     : '../experiments/final_ensemble/',
        'hop_size_fraction'  : 2,
    },
    {
        'thrs'               : [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
        'runs'               : [2],
        'tfs'                : ['tf_pcen_biodiversity'],
        'adaptive_embedding' : True,
        'stats_group'        : 0,
        'label'              : 'Prediction 2',
        'experiment_dir'     : '../experiments/final_ensemble/',
        'hop_size_fraction'  : 2,
    },
    {
        'thrs'               : [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
        'runs'               : [3],
        'tfs'                : ['tf_pcen_biodiversity'],
        'adaptive_embedding' : True,
        'stats_group'        : 0,
        'label'              : 'Prediction 3',
        'experiment_dir'     : '../experiments/final_ensemble/',
        'hop_size_fraction'  : 2,
    },
    {
        'thrs'               : [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
        'runs'               : [4],
        'tfs'                : ['tf_pcen_biodiversity'],
        'adaptive_embedding' : True,
        'stats_group'        : 0,
        'label'              : 'Prediction 4',
        'experiment_dir'     : '../experiments/final_ensemble/',
        'hop_size_fraction'  : 2,
    },

]

evaluate_and_plot(configs, csv_paths, "figure_2.pdf")

# Figure 3

In [ ]:
sample_rate = 22050
root_dir = '/mnt/storage_1/datasets/bioacoustics_dcase2022/Development_Set_{}Hz/Validation_Set/'.format(sample_rate)
csv_paths_me = glob.glob(os.path.join(root_dir, 'ME/*.csv'))
csv_paths_pb = glob.glob(os.path.join(root_dir, 'PB/*.csv'))
csv_paths_hb = glob.glob(os.path.join(root_dir, 'HB/*.csv'))
csv_paths = csv_paths_me + csv_paths_hb + csv_paths_pb

configs = [
    {
        'thrs'               : [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
        'runs'               : [0,1,2,3,4],
        'tfs'                : ['tf_decibel', 'tf_pcen_biodiversity', 'tf_pcen_speech'],
        'adaptive_embedding' : True,
        'stats_group'        : 0,
        'label'              : 'Ensemble',
        'experiment_dir'     : '../experiments/final_ensemble/',
        'hop_size_fraction'  : 2,
    },
    {
        'thrs'               : [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
        'runs'               : [0,1,2,3,4],
        'tfs'                : ['tf_decibel'],
        'adaptive_embedding' : True,
        'stats_group'        : 1,
        'label'              : 'log Mel',
        'experiment_dir'     : '../experiments/final_ensemble/',
        'hop_size_fraction'  : 2,
    },
    {
        'thrs'               : [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
        'runs'               : [0,1,2,3,4],
        'tfs'                : ['tf_pcen_biodiversity'],
        'adaptive_embedding' : True,
        'stats_group'        : 1,
        'label'              : 'PCEN (bio)',
        'experiment_dir'     : '../experiments/final_ensemble/',
        'hop_size_fraction'  : 2,
    },
    {
        'thrs'               : [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
        'runs'               : [0,1,2,3,4],
        'tfs'                : ['tf_pcen_speech'],
        'adaptive_embedding' : True,
        'stats_group'        : 1,
        'label'              : 'PCEN (spch)',
        'experiment_dir'     : '../experiments/final_ensemble/',
        'hop_size_fraction'  : 2,

    }
]

evaluate_and_plot(configs, csv_paths, "figure_3.pdf")

# Figure 4

In [ ]:
sample_rate = 22050
root_dir = '/mnt/storage_1/datasets/bioacoustics_dcase2022/Development_Set_{}Hz/Validation_Set/'.format(sample_rate)
csv_paths_me = glob.glob(os.path.join(root_dir, 'ME/*.csv'))
csv_paths_pb = glob.glob(os.path.join(root_dir, 'PB/*.csv'))
csv_paths_hb = glob.glob(os.path.join(root_dir, 'HB/*.csv'))
csv_paths = csv_paths_me + csv_paths_hb + csv_paths_pb

configs = [
    {
        'thrs'               : [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
        'runs'               : [0,1,2,3,4],
        'tfs'                : ['tf_decibel', 'tf_pcen_biodiversity', 'tf_pcen_speech'],
        'adaptive_embedding' : True,
        'stats_group'        : 0,
        'label'              : 'Adaptive',
        'experiment_dir'     : '../experiments/final_ensemble/',
        'hop_size_fraction'  : 2,

    },
    {
        'thrs'               : [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
        'runs'               : [0,1,2,3,4],
        'tfs'                : ['tf_decibel', 'tf_pcen_biodiversity', 'tf_pcen_speech'],
        'adaptive_embedding' : False,
        'window_size'        : 2048,
        'stats_group'        : 0,
        'label'              : 'T/sr = {:.2f} [s]'.format(2048/22050),
        'experiment_dir'     : '../experiments/final_ensemble/',
        'hop_size_fraction'  : 2,

    },
    {
        'thrs'               : [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
        'runs'               : [0,1,2,3,4],
        'tfs'                : ['tf_decibel', 'tf_pcen_biodiversity', 'tf_pcen_speech'],
        'adaptive_embedding' : False,
        'window_size'        : 4096,
        'stats_group'        : 0,
        'label'              : 'T/sr = {:.2f} [s]'.format(4096/22050),
        'experiment_dir'     : '../experiments/final_ensemble/',
        'hop_size_fraction'  : 2,

    },
    {
        'thrs'               : [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
        'runs'               : [0,1,2,3,4],
        'tfs'                : ['tf_decibel', 'tf_pcen_biodiversity', 'tf_pcen_speech'],
        'adaptive_embedding' : False,
        'window_size'        : 8192,
        'stats_group'        : 0,
        'label'              : 'T/sr = {:.2f} [s]'.format(8192/22050),
        'experiment_dir'     : '../experiments/final_ensemble/',
        'hop_size_fraction'  : 2,

    },
    {
        'thrs'               : [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
        'runs'               : [0,1,2,3,4],
        'tfs'                : ['tf_decibel', 'tf_pcen_biodiversity', 'tf_pcen_speech'],
        'adaptive_embedding' : False,
        'window_size'        : 16384,
        'stats_group'        : 0,
        'label'              : 'T/sr = {:.2f} [s]'.format(16384/22050),
        'experiment_dir'     : '../experiments/final_ensemble/',
        'hop_size_fraction'  : 2,

    },
]

evaluate_and_plot(configs, csv_paths, "figure_4.pdf")

# Final evaluation submissions

In [ ]:
root_dir = '/mnt/storage_1/datasets/bioacoustics_dcase2022/Evaluation_Set_22050Hz/'
#root_dir = '/mnt/storage_1/datasets/bioacoustics_dcase2022/Development_Set_22050Hz/Validation_Set/'

run_evaluation = False
csv_paths = glob.glob(os.path.join(root_dir, '*/*.csv'))

martinsson_1 = {
    'thrs'               : [0.6],
    'runs'               : [0,1,2,3,4],
    'tfs'                : ['tf_decibel', 'tf_pcen_biodiversity', 'tf_pcen_speech'],
    'adaptive_embedding' : True,
    'stats_group'        : 0,
    'label'              : 'Ensemble all TF',
    'experiment_dir'     : '../experiments/final_ensemble/',
    'hop_size_fraction'  : 2,
}

evaluate_and_plot([martinsson_1], csv_paths, "dummy.pdf", prediction_path='Martinsson_RISE_task5_1.output.csv', run_evaluation=run_evaluation)

martinsson_2 = {
    'thrs'               : [0.5],
    'runs'               : [0,1,2,3,4],
    'tfs'                : ['tf_decibel', 'tf_pcen_biodiversity', 'tf_pcen_speech'],
    'adaptive_embedding' : True,
    'stats_group'        : 0,
    'label'              : 'Ensemble all TF',
    'experiment_dir'     : '../experiments/final_ensemble/',
    'hop_size_fraction'  : 2,
}

evaluate_and_plot([martinsson_2], csv_paths, "dummy.pdf", prediction_path='Martinsson_RISE_task5_2.output.csv', run_evaluation=run_evaluation)

martinsson_3 = {
    'thrs'               : [0.6],
    'runs'               : [0],
    'tfs'                : ['tf_pcen_speech'],
    'adaptive_embedding' : True,
    'stats_group'        : 0,
    'label'              : 'Ensemble all TF',
    'experiment_dir'     : '../experiments/final_ensemble/',
    'hop_size_fraction'  : 2,
}

evaluate_and_plot([martinsson_3], csv_paths, "dummy.pdf", prediction_path='Martinsson_RISE_task5_3.output.csv', run_evaluation=run_evaluation)


martinsson_4 = {
    'thrs'               : [0.5],
    'runs'               : [0],
    'tfs'                : ['tf_pcen_speech'],
    'adaptive_embedding' : True,
    'stats_group'        : 0,
    'label'              : 'Ensemble all TF',
    'experiment_dir'     : '../experiments/final_ensemble/',
    'hop_size_fraction'  : 2,
}

evaluate_and_plot([martinsson_4], csv_paths, "dummy.pdf", prediction_path='Martinsson_RISE_task5_4.output.csv', run_evaluation=run_evaluation)

# Other results

In [ ]:
sample_rate = 22050
root_dir = '/mnt/storage_1/datasets/bioacoustics_dcase2022/Development_Set_{}Hz/Validation_Set/'.format(sample_rate)
csv_paths_me = glob.glob(os.path.join(root_dir, 'ME/*.csv'))
csv_paths_pb = glob.glob(os.path.join(root_dir, 'PB/*.csv'))
csv_paths_hb = glob.glob(os.path.join(root_dir, 'HB/*.csv'))
csv_paths = csv_paths_me + csv_paths_hb + csv_paths_pb

configs = [
    {
        'thrs'               : [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
        'runs'               : [0],
        'tfs'                : ['tf_pcen_speech'],
        'adaptive_embedding' : False,
        'window_size'        : 2048,
        'stats_group'        : 0,
        'label'              : 'Prediction 0',
        'experiment_dir'     : '../experiments/final_ensemble/',
        'hop_size_fraction'  : 2,

    },
    {
        'thrs'               : [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
        'runs'               : [1],
        'tfs'                : ['tf_pcen_speech'],
        'adaptive_embedding' : False,
        'window_size'        : 2048,
        'stats_group'        : 0,
        'label'              : 'Prediction 1',
        'experiment_dir'     : '../experiments/final_ensemble/',
        'hop_size_fraction'  : 2,

    },
    {
        'thrs'               : [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
        'runs'               : [2],
        'tfs'                : ['tf_pcen_speech'],
        'adaptive_embedding' : False,
        'window_size'        : 2048,
        'stats_group'        : 0,
        'label'              : 'Prediction 2',
        'experiment_dir'     : '../experiments/final_ensemble/',
        'hop_size_fraction'  : 2,

    },
    {
        'thrs'               : [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
        'runs'               : [3],
        'tfs'                : ['tf_pcen_speech'],
        'adaptive_embedding' : False,
        'window_size'        : 2048,
        'stats_group'        : 0,
        'label'              : 'Prediction 3',
        'experiment_dir'     : '../experiments/final_ensemble/',
        'hop_size_fraction'  : 2,

    },
    {
        'thrs'               : [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
        'runs'               : [4],
        'tfs'                : ['tf_pcen_speech'],
        'adaptive_embedding' : False,
        'window_size'        : 2048,
        'stats_group'        : 0,
        'label'              : 'Prediction 4',
        'experiment_dir'     : '../experiments/final_ensemble/',
        'hop_size_fraction'  : 2,

    },
]

evaluate_and_plot(configs)

In [ ]:
print(np.mean([0.4755088962166319,0.38637601736740684,0.42502901620372324,0.365077554509417,0.41393733991235127]))
print(np.std([0.4755088962166319,0.38637601736740684,0.42502901620372324,0.365077554509417,0.41393733991235127]))

In [ ]:
sample_rate = 22050
root_dir = '/mnt/storage_1/datasets/bioacoustics_dcase2022/Development_Set_{}Hz/Validation_Set/'.format(sample_rate)
csv_paths_me = glob.glob(os.path.join(root_dir, 'ME/*.csv'))
csv_paths_pb = glob.glob(os.path.join(root_dir, 'PB/*.csv'))
csv_paths_hb = glob.glob(os.path.join(root_dir, 'HB/*.csv'))
csv_paths = csv_paths_me + csv_paths_hb + csv_paths_pb

configs = [
    {
        'thrs'               : [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
        'runs'               : [0],
        'tfs'                : ['tf_pcen_speech'],
        'adaptive_embedding' : True,
        'window_size'        : 2048,
        'stats_group'        : 0,
        'label'              : 'Prediction 0',
        'experiment_dir'     : '../experiments/final_ensemble/',
        'hop_size_fraction'  : 2,

    },
    {
        'thrs'               : [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
        'runs'               : [1],
        'tfs'                : ['tf_pcen_speech'],
        'adaptive_embedding' : True,
        'window_size'        : 2048,
        'stats_group'        : 0,
        'label'              : 'Prediction 1',
        'experiment_dir'     : '../experiments/final_ensemble/',
        'hop_size_fraction'  : 2,

    },
    {
        'thrs'               : [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
        'runs'               : [2],
        'tfs'                : ['tf_pcen_speech'],
        'adaptive_embedding' : True,
        'window_size'        : 2048,
        'stats_group'        : 0,
        'label'              : 'Prediction 2',
        'experiment_dir'     : '../experiments/final_ensemble/',
        'hop_size_fraction'  : 2,

    },
    {
        'thrs'               : [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
        'runs'               : [3],
        'tfs'                : ['tf_pcen_speech'],
        'adaptive_embedding' : True,
        'window_size'        : 2048,
        'stats_group'        : 0,
        'label'              : 'Prediction 3',
        'experiment_dir'     : '../experiments/final_ensemble/',
        'hop_size_fraction'  : 2,

    },
    {
        'thrs'               : [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
        'runs'               : [4],
        'tfs'                : ['tf_pcen_speech'],
        'adaptive_embedding' : True,
        'window_size'        : 2048,
        'stats_group'        : 0,
        'label'              : 'Prediction 4',
        'experiment_dir'     : '../experiments/final_ensemble/',
        'hop_size_fraction'  : 2,

    },
]

evaluate_and_plot(configs, csv_paths)

In [ ]:
print(np.mean([0.5718666243983976, 0.47688408207269095, 0.5128513604816188, 0.40861540924728934, 0.5116620514059509]))
print(np.std([0.5718666243983976, 0.47688408207269095, 0.5128513604816188, 0.40861540924728934, 0.5116620514059509]))

In [ ]:
sample_rate = 22050
root_dir = '/mnt/storage_1/datasets/bioacoustics_dcase2022/Development_Set_{}Hz/Validation_Set/'.format(sample_rate)
csv_paths_me = glob.glob(os.path.join(root_dir, 'ME/*.csv'))
csv_paths_pb = glob.glob(os.path.join(root_dir, 'PB/*.csv'))
csv_paths_hb = glob.glob(os.path.join(root_dir, 'HB/*.csv'))
csv_paths = csv_paths_me + csv_paths_hb + csv_paths_pb

configs = [
    {
        'thrs'               : [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
        'runs'               : [0,1,2,3,4],
        'tfs'                : ['tf_decibel', 'tf_pcen_biodiversity', 'tf_pcen_speech'],
        'adaptive_embedding' : True,
        'stats_group'        : 0,
        'label'              : 'Ensemble (large)',
        'experiment_dir'     : '../experiments/final_ensemble/',
    },
    {
        'thrs'               : [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
        'runs'               : [0,1,2,3,4],
        'tfs'                : ['tf_decibel', 'tf_pcen_biodiversity', 'tf_pcen_speech'],
        'adaptive_embedding' : True,
        'stats_group'        : 0,
        'label'              : 'Ensemble (small)',
        'experiment_dir'     : '../experiments/final_ensemble_small/',
    },
]

evaluate_and_plot(configs)

In [ ]:
sample_rate = 22050
root_dir = '/mnt/storage_1/datasets/bioacoustics_dcase2022/Development_Set_{}Hz/Validation_Set/'.format(sample_rate)
csv_paths_me = glob.glob(os.path.join(root_dir, 'ME/*.csv'))
csv_paths_pb = glob.glob(os.path.join(root_dir, 'PB/*.csv'))
csv_paths_hb = glob.glob(os.path.join(root_dir, 'HB/*.csv'))
csv_paths = csv_paths_me + csv_paths_hb + csv_paths_pb

configs = [
    {
        'thrs'               : [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
        'runs'               : [0,1,2,3,4],
        'tfs'                : ['tf_decibel', 'tf_pcen_biodiversity', 'tf_pcen_speech'],
        'adaptive_embedding' : True,
        'stats_group'        : 0,
        'label'              : 'Ensemble',
        'experiment_dir'     : '../experiments/final_ensemble/',
        'hop_size_fraction'  : 2,
    },
    {
        'thrs'               : [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
        'runs'               : [0,1,2,3,4],
        'tfs'                : ['tf_decibel', 'tf_pcen_biodiversity', 'tf_pcen_speech'],
        'adaptive_embedding' : True,
        'stats_group'        : 0,
        'label'              : 'Ensemble (hop size small)',
        'experiment_dir'     : '../experiments/final_ensemble/',
        'hop_size_fraction'  : 4,
    },
]

evaluate_and_plot(configs)